In [43]:
import http.client
import json
import time
import csv
from datetime import datetime
from datetime import timedelta
import numpy as np
import pandas as pd
import re
import sys
if sys.version_info[0] < 3: 
    from StringIO import StringIO
else:
    from io import StringIO


class NRGStreamApi:    
    
    def __init__(self,username=None,password=None):
        self.username = 'Fahad'
        self.password = 'ABFAHc2'                
        self.server = 'api.nrgstream.com'        
        self.tokenPath = '/api/security/token'
        self.releasePath = '/api/ReleaseToken'
        self.tokenPayload = f'grant_type=password&username={self.username}&password={self.password}'
        self.tokenExpiry = datetime.now() - timedelta(seconds=60)
        self.accessToken = ""        
        
    def getToken(self):
        try:
            if self.isTokenValid() == False:                             
                headers = { }        
                # Connect to API server to get a token
                conn = http.client.HTTPSConnection(self.server)
                conn.request('POST', self.tokenPath, self.tokenPayload, headers)
                res = conn.getresponse()                
                res_code = res.code
                # Check if the response is good
                
                if res_code == 200:
                    res_data = res.read()
                    # Decode the token into an object
                    jsonData = json.loads(res_data.decode('utf-8'))
                    self.accessToken = jsonData['access_token']                         
                    # Calculate new expiry date
                    self.tokenExpiry = datetime.now() + timedelta(seconds=jsonData['expires_in'])                        
                    #print('token obtained')
                    #print(self.accessToken)
                else:
                    res_data = res.read()
                    print(res_data.decode('utf-8'))
                conn.close()                          
        except Exception as e:
            print("getToken: " + str(e))
            # Release token if an error occured
            self.releaseToken()      

    def releaseToken(self):
        try:            
            headers = {}
            headers['Authorization'] = f'Bearer {self.accessToken}'            
            conn = http.client.HTTPSConnection(self.server)
            conn.request('DELETE', self.releasePath, None, headers)  
            res = conn.getresponse()
            res_code = res.code
            if res_code == 200:   
                # Set expiration date back to guarantee isTokenValid() returns false                
                self.tokenExpiry = datetime.now() - timedelta(seconds=60)
                #print('token released')            
        except Exception as e:
            print("releaseToken: " + str(e))
                    
    def isTokenValid(self):
        if self.tokenExpiry==None:
            return False
        elif datetime.now() >= self.tokenExpiry:            
            return False
        else:
            return True            
    
    def GetStreamDataByStreamId(self,streamIds, fromDate, toDate, dataFormat='csv', dataOption=''):
        stream_data = "" 
        # Set file format to csv or json            
        DataFormats = {}
        DataFormats['csv'] = 'text/csv'
        DataFormats['json'] = 'Application/json'
        
        try:                            
            for streamId in streamIds:            
                # Get an access token            
                self.getToken()    
                if self.isTokenValid():
                    # Setup the path for data request. Pass dates in via function call
                    path = f'/api/StreamData/{streamId}'
                    if fromDate != '' and toDate != '':
                        path += f'?fromDate={fromDate.replace(" ", "%20")}&toDate={toDate.replace(" ", "%20")}'
                    if dataOption != '':
                        if fromDate != '' and toDate != '':
                            path += f'&dataOption={dataOption}'        
                        else:
                            path += f'?dataOption={dataOption}'        
                    
                    # Create request header
                    headers = {}            
                    headers['Accept'] = DataFormats[dataFormat]
                    headers['Authorization']= f'Bearer {self.accessToken}'
                    
                    # Connect to API server
                    conn = http.client.HTTPSConnection(self.server)
                    conn.request('GET', path, None, headers)
                    res = conn.getresponse()        
                    res_code = res.code                    
                    if res_code == 200:   
                        try:
                            print(f'{datetime.now()} Outputing stream {path} res code {res_code}')
                            # output return data to a text file            
                            if dataFormat == 'csv':
                                stream_data += res.read().decode('utf-8').replace('\r\n','\n') 
                            elif dataFormat == 'json':
                                stream_data += json.dumps(json.loads(res.read().decode('utf-8')), indent=2, sort_keys=False)
                            conn.close()

                        except Exception as e:
                            print(str(e))            
                            self.releaseToken()
                            return None  
                    else:
                        print(str(res_code) + " - " + str(res.reason) + " - " + str(res.read().decode('utf-8')))
                    
                self.releaseToken()   
                # Wait 1 second before next request
                time.sleep(1)
            return stream_data        
        except Exception as e:
            print(str(e))    
            self.releaseToken()
            return None
        
        
    def StreamDataOptions(self, streamId, dataFormat='csv'):
        try:      
            DataFormats = {}
            DataFormats['csv'] = 'text/csv'
            DataFormats['json'] = 'Application/json'
            resultSet = {}
            for streamId in streamIds:
                # Get an access token    
                if streamId not in resultSet:
                    self.getToken()                        
                    if self.isTokenValid():                 
                        # Setup the path for data request.
                        path = f'/api/StreamDataOptions/{streamId}'                        
                        # Create request header
                        headers = {}     
                        headers['Accept'] = DataFormats[dataFormat]                                   
                        headers['Authorization'] = f'Bearer {self.accessToken}'
                        # Connect to API server
                        conn = http.client.HTTPSConnection(self.server)
                        conn.request('GET', path, None, headers)
                        res = conn.getresponse()
                        self.releaseToken()       
                        if dataFormat == 'csv':
                            resultSet[streamId] = res.read().decode('utf-8').replace('\r\n','\n') 
                        elif dataFormat == 'json':
                            resultSet[streamId] = json.dumps(json.loads(res.read().decode('utf-8')), indent=2, sort_keys=False)                            
                    time.sleep(1)                        
            return resultSet            
        except Exception as e:
            print(str(e))    
            self.releaseToken()
            return None          
        
        except Exception as e:            
            self.releaseToken()                        
            return str(e)

        
# Authenticate with your NRG Stream username and password    
nrgStreamApi = NRGStreamApi('Username','Password')         
# Date range for your data request
# Date format must be mm/dd/yyyy
fromDate = '01/01/2019'
toDate =   '02/11/2021'
# Specify output format - 'csv' or 'json'
dataFormat = 'csv'
# Data Option
dataOption = ''
stream = [3]
for i in stream:
    nrgStreamApi = NRGStreamApi('Username','Password')
    ids= [i]
    stream_data = nrgStreamApi.GetStreamDataByStreamId(ids, fromDate, toDate, dataFormat, dataOption)        
    STREAM_DATA = StringIO(stream_data)
    df = pd.read_csv(STREAM_DATA, sep=";")

    
df = df[14:df.shape[0]]
df.columns = ["Datetime,AIL"]
new = df['Datetime,AIL'].str.split(",", n = 2, expand = True) 
# making separate first name column from new data frame 
df["Datetime"]= new[0] 
# making separate last name column from new data frame 
df["AIL"]= new[1] 
df['AIL'] = pd.to_numeric(df['AIL'],errors='coerce')
df['Datetime']= pd.to_datetime(df['Datetime'])
df= df.drop(columns=['Datetime,AIL'],axis=1)
df = df.reset_index(drop=True)
df

2021-02-15 17:59:20.221562 Outputing stream /api/StreamData/3?fromDate=01/01/2019&toDate=02/11/2021 res code 200


,Datetime,AIL
0,2019-01-01 00:00:00,10008.0
1,2019-01-01 01:00:00,9868.0
2,2019-01-01 02:00:00,9736.0
3,2019-01-01 03:00:00,9597.0
4,2019-01-01 04:00:00,9530.0
...,...,...
18523,2021-02-10 19:00:00,11485.0
18524,2021-02-10 20:00:00,11364.0
18525,2021-02-10 21:00:00,11195.0
18526,2021-02-10 22:00:00,10961.0


In [31]:
df['trend'] = df.index
df['trend'] = df['trend'] +1

###################
df['hour_of_day']= df['Datetime'].dt.hour

#------------------------------
'''

import numpy as np
# create a list of our conditions
conditions = [
    (df['hour_of_day'] < 7),
    (df['hour_of_day'] >= 7) & (df['hour_of_day'] <= 19),
    (df['hour_of_day'] > 19)
    ]


# create a list of the values we want to assign for each condition
values = [1, 0, 1]

# create a new column and use np.select to assign values to it using our lists as arguments
df['off_peak'] = np.select(conditions, values)

conditions = [
    (df['hour_of_day'] < 7),
    (df['hour_of_day'] >= 7) & (df['hour_of_day'] <= 19),
    (df['hour_of_day'] > 19)
    ]

# create a list of the values we want to assign for each condition
values = [0, 1, 0]

# create a new column and use np.select to assign values to it using our lists as arguments
df['on_peak'] = np.select(conditions, values)

'''


#----------------------
df['just_date'] = df['Datetime'].dt.date
dates = df['just_date']
day = pd.Series([d.timetuple().tm_yday for d in dates])
df['day'] = day

#---------

df['hour_x_day'] = df['day']*df['hour_of_day'] 

#------------------------

df['sin.day']   = np.sin(day*2*np.pi/365 )
df['cos.day']   = np.cos(day*2*np.pi/365 )

df['sin.hour']  = np.sin(df['hour_of_day']*2*np.pi/24)
df['cos.hour']  = np.cos(df['hour_of_day']*2*np.pi/24)

df['sin.trend']  = np.sin(df['trend']*2*np.pi/168)
df['cos.trend']  = np.cos(df['trend']*2*np.pi/168)
#-------------------------

weekdays = [d.weekday() for d in dates]
df['weekend'] = [1 if d >= 5 else 0 for d in weekdays]




#-----------------------------------------
'''

for i, s in enumerate(['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday',
            'sunday']):
    df[s] = [1 if d == i else 0 for d in weekdays]
    
    
from sklearn.preprocessing import OneHotEncoder
month = [[d.timetuple().tm_mon-1] for d in dates]
month_bin = OneHotEncoder(dtype=int, sparse=False).fit_transform(month)
for i in range(12):
    df['month_%d' % i] = month_bin[:, i]
    
for i in range(24):
    df['hour_%d' % i] = np.where(df['hour_of_day']==i, 1, 0)

'''
df['month'] = df['Datetime'].dt.month
    
df

,Datetime,AIL,trend,hour_of_day,just_date,day,hour_x_day,sin.day,cos.day,sin.hour,cos.hour,sin.trend,cos.trend,weekend,month
0,2019-01-01 00:00:00,10008.0,1,0,2019-01-01,1,0,0.017213,0.999852,0.000000,1.000000,0.037391,0.999301,0,1
1,2019-01-01 01:00:00,9868.0,2,1,2019-01-01,1,1,0.017213,0.999852,0.258819,0.965926,0.074730,0.997204,0,1
2,2019-01-01 02:00:00,9736.0,3,2,2019-01-01,1,2,0.017213,0.999852,0.500000,0.866025,0.111964,0.993712,0,1
3,2019-01-01 03:00:00,9597.0,4,3,2019-01-01,1,3,0.017213,0.999852,0.707107,0.707107,0.149042,0.988831,0,1
4,2019-01-01 04:00:00,9530.0,5,4,2019-01-01,1,4,0.017213,0.999852,0.866025,0.500000,0.185912,0.982566,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18523,2021-02-10 19:00:00,11485.0,18524,19,2021-02-10,41,779,0.648630,0.761104,-0.965926,0.258819,0.997204,-0.074730,0,2
18524,2021-02-10 20:00:00,11364.0,18525,20,2021-02-10,41,820,0.648630,0.761104,-0.866025,0.500000,0.993712,-0.111964,0,2
18525,2021-02-10 21:00:00,11195.0,18526,21,2021-02-10,41,861,0.648630,0.761104,-0.707107,0.707107,0.988831,-0.149042,0,2
18526,2021-02-10 22:00:00,10961.0,18527,22,2021-02-10,41,902,0.648630,0.761104,-0.500000,0.866025,0.982566,-0.185912,0,2


In [32]:
xls = pd.ExcelFile('edmonton_sunrise_sunset.xls')
dfess_2015 = pd.read_excel(xls, '2015') #df= dafaframe, e= edmonston, ss= sunrise and sunset 
dfess_2016 = pd.read_excel(xls, '2016')

df['year']= df['Datetime'].dt.year
dfess_2015 = dfess_2015[['Sunrise_hr', 'Sunset_hr']]
dfess_2016 = dfess_2016[['Sunrise_hr', 'Sunset_hr']]
df['sunlight_avaialbility'] =  0

for i in range(0,df.shape[0]):
    if (df.iloc[i,df.columns.get_loc('year')]%4 ==0 and df.iloc[i,df.columns.get_loc('year')]%100 !=0):
        criteria = df.iloc[i,df.columns.get_loc('day')] #day_of_year
        sunrise  = dfess_2016.iloc[criteria-1,0] #sunrise
        sunset   = dfess_2016.iloc[criteria-1,1] #sunset
        if (df.iloc[i,df.columns.get_loc('hour_of_day')]>= sunrise) and (df.iloc[i,df.columns.get_loc('hour_of_day')] <= sunset):
            df.iloc[i,df.columns.get_loc('sunlight_avaialbility')] = 1
    else:
        criteria = df.iloc[i,df.columns.get_loc('day')] #day_of_year
        sunrise  = dfess_2015.iloc[criteria-1,0]#sunrise
        sunset   = dfess_2015.iloc[criteria-1,1] #sunset
        if (df.iloc[i,df.columns.get_loc('hour_of_day')]>= sunrise) and (df.iloc[i,df.columns.get_loc('hour_of_day')] <= sunset):
            df.iloc[i,df.columns.get_loc('sunlight_avaialbility')] = 1
            
df

,Datetime,AIL,trend,hour_of_day,just_date,day,hour_x_day,sin.day,cos.day,sin.hour,cos.hour,sin.trend,cos.trend,weekend,month,year,sunlight_avaialbility
0,2019-01-01 00:00:00,10008.0,1,0,2019-01-01,1,0,0.017213,0.999852,0.000000,1.000000,0.037391,0.999301,0,1,2019,0
1,2019-01-01 01:00:00,9868.0,2,1,2019-01-01,1,1,0.017213,0.999852,0.258819,0.965926,0.074730,0.997204,0,1,2019,0
2,2019-01-01 02:00:00,9736.0,3,2,2019-01-01,1,2,0.017213,0.999852,0.500000,0.866025,0.111964,0.993712,0,1,2019,0
3,2019-01-01 03:00:00,9597.0,4,3,2019-01-01,1,3,0.017213,0.999852,0.707107,0.707107,0.149042,0.988831,0,1,2019,0
4,2019-01-01 04:00:00,9530.0,5,4,2019-01-01,1,4,0.017213,0.999852,0.866025,0.500000,0.185912,0.982566,0,1,2019,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18523,2021-02-10 19:00:00,11485.0,18524,19,2021-02-10,41,779,0.648630,0.761104,-0.965926,0.258819,0.997204,-0.074730,0,2,2021,0
18524,2021-02-10 20:00:00,11364.0,18525,20,2021-02-10,41,820,0.648630,0.761104,-0.866025,0.500000,0.993712,-0.111964,0,2,2021,0
18525,2021-02-10 21:00:00,11195.0,18526,21,2021-02-10,41,861,0.648630,0.761104,-0.707107,0.707107,0.988831,-0.149042,0,2,2021,0
18526,2021-02-10 22:00:00,10961.0,18527,22,2021-02-10,41,902,0.648630,0.761104,-0.500000,0.866025,0.982566,-0.185912,0,2,2021,0


In [33]:
df['AIL_previous_hour']  = df['AIL'].shift(1)    
df['AIL_24h_lagged']     = df['AIL'].shift(24)    
df['AIL_2day_lagged']    = df['AIL'].shift(24*2)  
df['AIL_3day_lagged']    = df['AIL'].shift(24*3)  
df['AIL_4day_lagged']    = df['AIL'].shift(24*4)  
df['AIL_5day_lagged']    = df['AIL'].shift(24*5)  
df['AIL_6day_lagged']    = df['AIL'].shift(24*6)  
df['AIL_oneweek_lagged'] = df['AIL'].shift(24*7)  
df

,Datetime,AIL,trend,hour_of_day,just_date,day,hour_x_day,sin.day,cos.day,sin.hour,...,year,sunlight_avaialbility,AIL_previous_hour,AIL_24h_lagged,AIL_2day_lagged,AIL_3day_lagged,AIL_4day_lagged,AIL_5day_lagged,AIL_6day_lagged,AIL_oneweek_lagged
0,2019-01-01 00:00:00,10008.0,1,0,2019-01-01,1,0,0.017213,0.999852,0.000000,...,2019,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2019-01-01 01:00:00,9868.0,2,1,2019-01-01,1,1,0.017213,0.999852,0.258819,...,2019,0,10008.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-01-01 02:00:00,9736.0,3,2,2019-01-01,1,2,0.017213,0.999852,0.500000,...,2019,0,9868.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2019-01-01 03:00:00,9597.0,4,3,2019-01-01,1,3,0.017213,0.999852,0.707107,...,2019,0,9736.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-01-01 04:00:00,9530.0,5,4,2019-01-01,1,4,0.017213,0.999852,0.866025,...,2019,0,9597.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18523,2021-02-10 19:00:00,11485.0,18524,19,2021-02-10,41,779,0.648630,0.761104,-0.965926,...,2021,0,11665.0,11598.0,11513.0,11271.0,11326.0,11298.0,11165.0,10961.0
18524,2021-02-10 20:00:00,11364.0,18525,20,2021-02-10,41,820,0.648630,0.761104,-0.866025,...,2021,0,11485.0,11490.0,11390.0,11127.0,11235.0,11178.0,11057.0,10887.0
18525,2021-02-10 21:00:00,11195.0,18526,21,2021-02-10,41,861,0.648630,0.761104,-0.707107,...,2021,0,11364.0,11410.0,11283.0,11093.0,11181.0,11018.0,10860.0,10725.0
18526,2021-02-10 22:00:00,10961.0,18527,22,2021-02-10,41,902,0.648630,0.761104,-0.500000,...,2021,0,11195.0,11192.0,11148.0,10958.0,11030.0,10811.0,10590.0,10524.0


In [34]:
# Authenticate with your NRG Stream username and password    
nrgStreamApi = NRGStreamApi('Username','Password')         
# Date range for your data request
# Date format must be mm/dd/yyyy
fromDate = '12/31/2018'
toDate =   '12/31/2018'
# Specify output format - 'csv' or 'json'
dataFormat = 'csv'
# Data Option
dataOption = ''
stream = [3]
for i in stream:
    nrgStreamApi = NRGStreamApi('Username','Password')
    ids= [i]
    stream_data = nrgStreamApi.GetStreamDataByStreamId(ids, fromDate, toDate, dataFormat, dataOption)        
    STREAM_DATA = StringIO(stream_data)
    test_df = pd.read_csv(STREAM_DATA, sep=";")

    
test_df = test_df[14:test_df.shape[0]]
test_df.columns = ["Datetime,AIL"]
new = test_df['Datetime,AIL'].str.split(",", n = 2, expand = True) 
# making separate first name column from new data frame 
test_df["Datetime"]= new[0] 
# making separate last name column from new data frame 
test_df["AIL"]= new[1] 
test_df['AIL'] = pd.to_numeric(df['AIL'],errors='coerce')
test_df['Datetime']= pd.to_datetime(df['Datetime'])
test_df= test_df.drop(columns=['Datetime,AIL'],axis=1)
test_df = test_df.reset_index(drop=True)

df.iloc[0,df.columns.get_loc('AIL_previous_hour')] = test_df.iloc[23,1]

df.iloc[:,df.columns.get_loc('AIL_previous_hour')]

2021-02-13 21:46:11.546583 Outputing stream /api/StreamData/3?fromDate=12/31/2018&toDate=12/31/2018 res code 200


0        10484.0
1        10008.0
2         9868.0
3         9736.0
4         9597.0
          ...   
18523    11665.0
18524    11485.0
18525    11364.0
18526    11195.0
18527    10961.0
Name: AIL_previous_hour, Length: 18528, dtype: float64

In [35]:
# Authenticate with your NRG Stream username and password    
nrgStreamApi = NRGStreamApi('Username','Password')         
# Date range for your data request
# Date format must be mm/dd/yyyy
fromDate = '01/01/2019'
toDate =   '01/08/2019'
# Specify output format - 'csv' or 'json'
dataFormat = 'csv'
# Data Option
dataOption = ''
stream = [3]
for i in stream:
    nrgStreamApi = NRGStreamApi('Username','Password')
    ids= [i]
    stream_data = nrgStreamApi.GetStreamDataByStreamId(ids, fromDate, toDate, dataFormat, dataOption)        
    STREAM_DATA = StringIO(stream_data)
    test_df = pd.read_csv(STREAM_DATA, sep=";")
    #test_df = test_df.iloc[1:14+7*24+1]
    #test_df = test_df[0: 183]
    test_df.columns = ["Datetime,AIL"]
    new = test_df['Datetime,AIL'].str.split(",", n = 2, expand = True) 
    # making separate first name column from new data frame 
    test_df["Datetime"]= new[0] 
    # making separate last name column from new data frame 
    test_df["AIL"]= new[1] 
    test_df['AIL'] = pd.to_numeric(df['AIL'],errors='coerce')
    test_df['Datetime']= pd.to_datetime(df['Datetime'])
    test_df= test_df.drop(columns=['Datetime,AIL'],axis=1)
    test_df = test_df.reset_index(drop=True)

#test_df
df.iloc[0:7*24, df.columns.get_loc('AIL_oneweek_lagged')] = test_df.iloc[0:24*7,1]
df.iloc[0:6*24, df.columns.get_loc('AIL_6day_lagged')]    = test_df.iloc[0:24*6,1]
df.iloc[0:5*24, df.columns.get_loc('AIL_5day_lagged')]    = test_df.iloc[0:24*5,1]
df.iloc[0:4*24, df.columns.get_loc('AIL_4day_lagged')]    = test_df.iloc[0:24*4,1]
df.iloc[0:3*24, df.columns.get_loc('AIL_3day_lagged')]    = test_df.iloc[0:24*3,1]
df.iloc[0:2*24, df.columns.get_loc('AIL_2day_lagged')]    = test_df.iloc[0:24*2,1]
df.iloc[0:1*24, df.columns.get_loc('AIL_24h_lagged')]     = test_df.iloc[0:24*1,1]

2021-02-13 21:46:21.436413 Outputing stream /api/StreamData/3?fromDate=01/01/2019&toDate=01/08/2019 res code 200


In [36]:
df1 = pd.DataFrame(df)
df1

,Datetime,AIL,trend,hour_of_day,just_date,day,hour_x_day,sin.day,cos.day,sin.hour,...,year,sunlight_avaialbility,AIL_previous_hour,AIL_24h_lagged,AIL_2day_lagged,AIL_3day_lagged,AIL_4day_lagged,AIL_5day_lagged,AIL_6day_lagged,AIL_oneweek_lagged
0,2019-01-01 00:00:00,10008.0,1,0,2019-01-01,1,0,0.017213,0.999852,0.000000,...,2019,0,10484.0,10008.0,10008.0,10008.0,10008.0,10008.0,10008.0,10008.0
1,2019-01-01 01:00:00,9868.0,2,1,2019-01-01,1,1,0.017213,0.999852,0.258819,...,2019,0,10008.0,9868.0,9868.0,9868.0,9868.0,9868.0,9868.0,9868.0
2,2019-01-01 02:00:00,9736.0,3,2,2019-01-01,1,2,0.017213,0.999852,0.500000,...,2019,0,9868.0,9736.0,9736.0,9736.0,9736.0,9736.0,9736.0,9736.0
3,2019-01-01 03:00:00,9597.0,4,3,2019-01-01,1,3,0.017213,0.999852,0.707107,...,2019,0,9736.0,9597.0,9597.0,9597.0,9597.0,9597.0,9597.0,9597.0
4,2019-01-01 04:00:00,9530.0,5,4,2019-01-01,1,4,0.017213,0.999852,0.866025,...,2019,0,9597.0,9530.0,9530.0,9530.0,9530.0,9530.0,9530.0,9530.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18523,2021-02-10 19:00:00,11485.0,18524,19,2021-02-10,41,779,0.648630,0.761104,-0.965926,...,2021,0,11665.0,11598.0,11513.0,11271.0,11326.0,11298.0,11165.0,10961.0
18524,2021-02-10 20:00:00,11364.0,18525,20,2021-02-10,41,820,0.648630,0.761104,-0.866025,...,2021,0,11485.0,11490.0,11390.0,11127.0,11235.0,11178.0,11057.0,10887.0
18525,2021-02-10 21:00:00,11195.0,18526,21,2021-02-10,41,861,0.648630,0.761104,-0.707107,...,2021,0,11364.0,11410.0,11283.0,11093.0,11181.0,11018.0,10860.0,10725.0
18526,2021-02-10 22:00:00,10961.0,18527,22,2021-02-10,41,902,0.648630,0.761104,-0.500000,...,2021,0,11195.0,11192.0,11148.0,10958.0,11030.0,10811.0,10590.0,10524.0


In [37]:
fromDate ='01/01/2019'
toDate   ='02/11/2021'

df_clgry   = pd.DataFrame()
df_edmtn   = pd.DataFrame()
df_ftmcmry = pd.DataFrame()
df_lthbrg  = pd.DataFrame()
df_mdcnht  = pd.DataFrame()
df_rddr    = pd.DataFrame()
df_slvlk   = pd.DataFrame()
df_list = {}

streams = [242498, 242497, 242500, 242508, 242511, 242519, 242522]
city_names = ['calgary', 'edmonton', 'ftmcmry','lthbrg','mdcnht', 'rddr','slvlk']
k = 0

for i in streams:
    stream_data = nrgStreamApi.GetStreamDataByStreamId([i], fromDate, toDate, 'csv', '')        
    STREAM_DATA = StringIO(stream_data)
    df = pd.read_csv(STREAM_DATA, sep=";")
    length = df.shape[0]
    df = df[16:length] # removing header information
    df.columns = ["Datetime,temp,wind,direction"]
    new = df['Datetime,temp,wind,direction'].str.split(",", n = 4, expand = True) 
    # making separate first name column from new data frame 
    df["Datetime"]= new[0] 
    df["temp"]= new[1] 
    df["wind"] = new [2]
    df["direction"] = new[3]
    df = df.drop(['Datetime,temp,wind,direction','direction'],axis=1)
    df = df.reset_index(drop=True)
    df.columns = [str(col) + '_'+ city_names[k] for col in df.columns]
    df_list[k] = df
    k = k+1

df_clgry   = df_list[0]
df_edmtn   = df_list[1]
df_ftmcmry = df_list[2]
df_lthbrg  = df_list[3]
df_mdcnht  = df_list[4]
df_rddr    = df_list[5]
df_slvlk   = df_list[6]


alberta_weather_merged = pd.concat([df_clgry, df_edmtn, df_ftmcmry, 
                                   df_lthbrg, df_mdcnht,
                                   df_rddr, df_slvlk], axis=1)

alberta_weather_merged = alberta_weather_merged[['Datetime_calgary', 'temp_calgary', 'wind_calgary',
       'temp_edmonton', 'wind_edmonton', 'temp_ftmcmry', 'wind_ftmcmry',
       'temp_lthbrg', 'wind_lthbrg', 'temp_mdcnht', 'wind_mdcnht', 'temp_rddr',
       'wind_rddr', 'temp_slvlk', 'wind_slvlk']]
alberta_weather_merged

2021-02-13 21:46:56.246666 Outputing stream /api/StreamData/242498?fromDate=01/01/2019&toDate=02/11/2021 res code 200
2021-02-13 21:47:00.543831 Outputing stream /api/StreamData/242497?fromDate=01/01/2019&toDate=02/11/2021 res code 200
2021-02-13 21:47:04.643474 Outputing stream /api/StreamData/242500?fromDate=01/01/2019&toDate=02/11/2021 res code 200
2021-02-13 21:47:09.251741 Outputing stream /api/StreamData/242508?fromDate=01/01/2019&toDate=02/11/2021 res code 200
2021-02-13 21:47:12.733990 Outputing stream /api/StreamData/242511?fromDate=01/01/2019&toDate=02/11/2021 res code 200
2021-02-13 21:47:16.420167 Outputing stream /api/StreamData/242519?fromDate=01/01/2019&toDate=02/11/2021 res code 200
2021-02-13 21:47:20.106817 Outputing stream /api/StreamData/242522?fromDate=01/01/2019&toDate=02/11/2021 res code 200


,Datetime_calgary,temp_calgary,wind_calgary,temp_edmonton,wind_edmonton,temp_ftmcmry,wind_ftmcmry,temp_lthbrg,wind_lthbrg,temp_mdcnht,wind_mdcnht,temp_rddr,wind_rddr,temp_slvlk,wind_slvlk
0,May 22 2019 00:00,7,10,12,10,13,10,7,10,8,10,7,10,12,10
1,May 22 2019 01:00,7,10,11,10,12,10,6,10,8,10,7,10,11,10
2,May 22 2019 02:00,6,10,11,10,11,10,6,10,7,10,6,10,11,10
3,May 22 2019 03:00,6,10,9,10,10,5,5,10,7,10,6,5,10,10
4,May 22 2019 04:00,6,10,9,10,9,5,5,10,7,10,5,5,9,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15139,Feb 10 2021 19:00,-28,20,-27,20,-29,20,-27,20,-27,20,-30,20,-26,10
15140,Feb 10 2021 20:00,-28,20,-28,20,-30,20,-28,20,-28,20,-31,20,-27,10
15141,Feb 10 2021 21:00,-29,10,-28,15,-31,10,-31,10,-29,10,-32,10,-28,10
15142,Feb 10 2021 22:00,-29,10,-29,15,-32,10,-31,10,-29,10,-33,10,-29,10


In [38]:
fromDate ='01/01/2019'
toDate   ='05/22/2019'

df_clgry   = pd.DataFrame()
df_edmtn   = pd.DataFrame()
df_ftmcmry = pd.DataFrame()
df_lthbrg  = pd.DataFrame()
df_mdcnht  = pd.DataFrame()
df_rddr    = pd.DataFrame()
df_slvlk   = pd.DataFrame()
df_list = {}

streams = [5, 11, 2320, 13, 2326, 4060, 2327]
city_names = ['calgary', 'edmonton', 'ftmcmry','lthbrg','mdcnht', 'rddr','slvlk']
k = 0

for i in streams:
    stream_data = nrgStreamApi.GetStreamDataByStreamId([i], fromDate, toDate, 'csv', '')        
    STREAM_DATA = StringIO(stream_data)
    df = pd.read_csv(STREAM_DATA, sep=";")
    length = df.shape[0]
    df = df[16:length] # removing header information
    df.columns = ["Datetime,temp,wind,direction"]
    new = df['Datetime,temp,wind,direction'].str.split(",", n = 4, expand = True) 
    # making separate first name column from new data frame 
    df["Datetime"]= new[0] 
    df["temp"]= new[1] 
    df["wind"] = new [2]
    df["direction"] = new[3]
    df = df.drop(['Datetime,temp,wind,direction','direction'],axis=1)
    df = df.reset_index(drop=True)
    df.columns = [str(col) + '_'+ city_names[k] for col in df.columns]
    df_list[k] = df
    k = k+1

df_clgry   = df_list[0]
df_edmtn   = df_list[1]
df_ftmcmry = df_list[2]
df_lthbrg  = df_list[3]
df_mdcnht  = df_list[4]
df_rddr    = df_list[5]
df_slvlk   = df_list[6]


alberta_weather_merged_2 = pd.concat([df_clgry, df_edmtn, df_ftmcmry, 
                                   df_lthbrg, df_mdcnht,
                                   df_rddr, df_slvlk], axis=1)

alberta_weather_merged_2 = alberta_weather_merged_2[['Datetime_calgary', 'temp_calgary', 'wind_calgary',
       'temp_edmonton', 'wind_edmonton', 'temp_ftmcmry', 'wind_ftmcmry',
       'temp_lthbrg', 'wind_lthbrg', 'temp_mdcnht', 'wind_mdcnht', 'temp_rddr',
       'wind_rddr', 'temp_slvlk', 'wind_slvlk']]
alberta_weather_merged_2

2021-02-13 21:47:34.955890 Outputing stream /api/StreamData/5?fromDate=01/01/2019&toDate=05/22/2019 res code 200
2021-02-13 21:47:37.874946 Outputing stream /api/StreamData/11?fromDate=01/01/2019&toDate=05/22/2019 res code 200
2021-02-13 21:47:40.257309 Outputing stream /api/StreamData/2320?fromDate=01/01/2019&toDate=05/22/2019 res code 200
2021-02-13 21:47:42.843108 Outputing stream /api/StreamData/13?fromDate=01/01/2019&toDate=05/22/2019 res code 200
2021-02-13 21:47:46.369654 Outputing stream /api/StreamData/2326?fromDate=01/01/2019&toDate=05/22/2019 res code 200
2021-02-13 21:47:50.049848 Outputing stream /api/StreamData/4060?fromDate=01/01/2019&toDate=05/22/2019 res code 200
2021-02-13 21:47:52.471065 Outputing stream /api/StreamData/2327?fromDate=01/01/2019&toDate=05/22/2019 res code 200


,Datetime_calgary,temp_calgary,wind_calgary,temp_edmonton,wind_edmonton,temp_ftmcmry,wind_ftmcmry,temp_lthbrg,wind_lthbrg,temp_mdcnht,wind_mdcnht,temp_rddr,wind_rddr,temp_slvlk,wind_slvlk
0,Jan 01 2019 00:00,-11.1,7,-9.8,13,-18.1,11,-11.8,,-12.4,,-15.2,11,-14.4,8
1,Jan 01 2019 01:00,-8.7,7,-9.3,14,-17,9,-10.7,,-12.5,,-13.3,5,-13.6,5
2,Jan 01 2019 02:00,-7.4,6,-8.2,14,-15.2,8,-8.5,,-11.8,,-11.8,13,-12.9,5
3,Jan 01 2019 03:00,-4.9,8,-7.3,14,-13.2,12,-6.7,,-10.6,,-10.2,14,-14.1,5
4,Jan 01 2019 04:00,-0.9,4,-6.3,11,-11.5,9,-3.6,,-8.1,,-8.8,17,-12.2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3379,May 21 2019 19:00,6.8,10,18.3,8,21.9,6,6.7,15,10.4,12,11.4,13,20.2,21
3380,May 21 2019 20:00,6.6,15,17.2,8,20,8,6.4,9,10,8,11.2,7,18.5,15
3381,May 21 2019 21:00,6.2,13,15.7,8,15.1,8,6.4,7,10,8,10.2,9,16.3,15
3382,May 21 2019 22:00,6.1,10,14.2,7,12.3,13,6.4,7,9.4,5,8,9,14.4,5


In [39]:
alberta_weather_merged_2 = alberta_weather_merged_2.append(alberta_weather_merged)
alberta_weather_merged_2.columns =['Datetime', 'temp_calgary', 'wind_calgary', 'temp_edmonton',
       'wind_edmonton', 'temp_ftmcmry', 'wind_ftmcmry', 'temp_lthbrg',
       'wind_lthbrg', 'temp_mdcnht', 'wind_mdcnht', 'temp_rddr', 'wind_rddr',
       'temp_slvlk', 'wind_slvlk'
]
alberta_weather_merged_2

,Datetime,temp_calgary,wind_calgary,temp_edmonton,wind_edmonton,temp_ftmcmry,wind_ftmcmry,temp_lthbrg,wind_lthbrg,temp_mdcnht,wind_mdcnht,temp_rddr,wind_rddr,temp_slvlk,wind_slvlk
0,Jan 01 2019 00:00,-11.1,7,-9.8,13,-18.1,11,-11.8,,-12.4,,-15.2,11,-14.4,8
1,Jan 01 2019 01:00,-8.7,7,-9.3,14,-17,9,-10.7,,-12.5,,-13.3,5,-13.6,5
2,Jan 01 2019 02:00,-7.4,6,-8.2,14,-15.2,8,-8.5,,-11.8,,-11.8,13,-12.9,5
3,Jan 01 2019 03:00,-4.9,8,-7.3,14,-13.2,12,-6.7,,-10.6,,-10.2,14,-14.1,5
4,Jan 01 2019 04:00,-0.9,4,-6.3,11,-11.5,9,-3.6,,-8.1,,-8.8,17,-12.2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15139,Feb 10 2021 19:00,-28,20,-27,20,-29,20,-27,20,-27,20,-30,20,-26,10
15140,Feb 10 2021 20:00,-28,20,-28,20,-30,20,-28,20,-28,20,-31,20,-27,10
15141,Feb 10 2021 21:00,-29,10,-28,15,-31,10,-31,10,-29,10,-32,10,-28,10
15142,Feb 10 2021 22:00,-29,10,-29,15,-32,10,-31,10,-29,10,-33,10,-29,10


In [40]:
#df['Datetime'] = pd.to_datetime(df['Datetime'])
alberta_weather_merged_2['Datetime'] = pd.to_datetime(alberta_weather_merged_2['Datetime'])
df = pd.merge(df1, alberta_weather_merged_2, on="Datetime")
df

,Datetime,AIL,trend,hour_of_day,just_date,day,hour_x_day,sin.day,cos.day,sin.hour,...,temp_ftmcmry,wind_ftmcmry,temp_lthbrg,wind_lthbrg,temp_mdcnht,wind_mdcnht,temp_rddr,wind_rddr,temp_slvlk,wind_slvlk
0,2019-01-01 00:00:00,10008.0,1,0,2019-01-01,1,0,0.017213,0.999852,0.000000,...,-18.1,11,-11.8,,-12.4,,-15.2,11,-14.4,8
1,2019-01-01 01:00:00,9868.0,2,1,2019-01-01,1,1,0.017213,0.999852,0.258819,...,-17,9,-10.7,,-12.5,,-13.3,5,-13.6,5
2,2019-01-01 02:00:00,9736.0,3,2,2019-01-01,1,2,0.017213,0.999852,0.500000,...,-15.2,8,-8.5,,-11.8,,-11.8,13,-12.9,5
3,2019-01-01 03:00:00,9597.0,4,3,2019-01-01,1,3,0.017213,0.999852,0.707107,...,-13.2,12,-6.7,,-10.6,,-10.2,14,-14.1,5
4,2019-01-01 04:00:00,9530.0,5,4,2019-01-01,1,4,0.017213,0.999852,0.866025,...,-11.5,9,-3.6,,-8.1,,-8.8,17,-12.2,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18523,2021-02-10 19:00:00,11485.0,18524,19,2021-02-10,41,779,0.648630,0.761104,-0.965926,...,-29,20,-27,20,-27,20,-30,20,-26,10
18524,2021-02-10 20:00:00,11364.0,18525,20,2021-02-10,41,820,0.648630,0.761104,-0.866025,...,-30,20,-28,20,-28,20,-31,20,-27,10
18525,2021-02-10 21:00:00,11195.0,18526,21,2021-02-10,41,861,0.648630,0.761104,-0.707107,...,-31,10,-31,10,-29,10,-32,10,-28,10
18526,2021-02-10 22:00:00,10961.0,18527,22,2021-02-10,41,902,0.648630,0.761104,-0.500000,...,-32,10,-31,10,-29,10,-33,10,-29,10


In [41]:
df = df.dropna()

In [42]:
df.to_csv("new_training_data_v7_2019_2020_feb_13_20201.csv")